- Main Function

In [104]:
rtRDD=sc.textFile("ratings.csv") \
    .map(lambda row: row.split(","))
rtRDD_Header=rtRDD.first()
rtRDD=rtRDD.filter(lambda x: x!=rtRDD_Header).map(lambda row: (row[0],row[1],float(row[2])))
rtRDD.collect()

[('1', '31', 2.5),
 ('1', '1029', 3.0),
 ('1', '1061', 3.0),
 ('1', '1129', 2.0),
 ('1', '1172', 4.0),
 ('1', '1263', 2.0),
 ('1', '1287', 2.0),
 ('1', '1293', 2.0),
 ('1', '1339', 3.5),
 ('1', '1343', 2.0),
 ('1', '1371', 2.5),
 ('1', '1405', 1.0),
 ('1', '1953', 4.0),
 ('1', '2105', 4.0),
 ('1', '2150', 3.0),
 ('1', '2193', 2.0),
 ('1', '2294', 2.0),
 ('1', '2455', 2.5),
 ('1', '2968', 1.0),
 ('1', '3671', 3.0),
 ('2', '10', 4.0),
 ('2', '17', 5.0),
 ('2', '39', 5.0),
 ('2', '47', 4.0),
 ('2', '50', 4.0),
 ('2', '52', 3.0),
 ('2', '62', 3.0),
 ('2', '110', 4.0),
 ('2', '144', 3.0),
 ('2', '150', 5.0),
 ('2', '153', 4.0),
 ('2', '161', 3.0),
 ('2', '165', 3.0),
 ('2', '168', 3.0),
 ('2', '185', 3.0),
 ('2', '186', 3.0),
 ('2', '208', 3.0),
 ('2', '222', 5.0),
 ('2', '223', 1.0),
 ('2', '225', 3.0),
 ('2', '235', 3.0),
 ('2', '248', 3.0),
 ('2', '253', 4.0),
 ('2', '261', 4.0),
 ('2', '265', 5.0),
 ('2', '266', 5.0),
 ('2', '272', 3.0),
 ('2', '273', 4.0),
 ('2', '292', 3.0),
 ('2', '2

In [105]:
givenMovieId = '31'
givenUserId = '1'
avgRating = 3.0

In [106]:
GoodRtUsRDD=rtRDD.filter(lambda row: row[2]>avgRating) \
     .filter(lambda row: row[1]==givenMovieId)\
     .map(lambda row: row[0]) \
     .distinct()
GoodRtUsLs=GoodRtUsRDD.collect() # Convert to python list
GoodRtUsLs

['73',
 '110',
 '111',
 '325',
 '341',
 '485',
 '496',
 '641',
 '31',
 '32',
 '165',
 '242',
 '487',
 '511',
 '516',
 '525',
 '607']

In [107]:
UserSltMvRDD=rtRDD.filter(lambda row: row[0] in GoodRtUsLs).map(lambda row: (row[1],row[2]))
UserSltMvRDD.collect()

[('31', 4.0),
 ('32', 4.5),
 ('50', 3.5),
 ('111', 4.5),
 ('260', 4.0),
 ('296', 4.5),
 ('318', 4.0),
 ('372', 3.5),
 ('379', 4.0),
 ('527', 5.0),
 ('541', 4.0),
 ('778', 3.5),
 ('858', 4.5),
 ('904', 4.0),
 ('1089', 4.5),
 ('1091', 3.0),
 ('1186', 2.0),
 ('1197', 4.5),
 ('1206', 4.5),
 ('1208', 4.5),
 ('1221', 4.5),
 ('1234', 4.5),
 ('1299', 4.0),
 ('1378', 4.0),
 ('1416', 3.0),
 ('1957', 4.5),
 ('1967', 4.5),
 ('2028', 4.5),
 ('2248', 4.0),
 ('2329', 4.5),
 ('2502', 4.0),
 ('2571', 4.5),
 ('2692', 4.5),
 ('2959', 5.0),
 ('2997', 4.5),
 ('3072', 3.0),
 ('3087', 3.5),
 ('3698', 4.5),
 ('3755', 4.0),
 ('3897', 4.0),
 ('3911', 4.0),
 ('4886', 4.0),
 ('4979', 3.5),
 ('4993', 4.5),
 ('5418', 4.5),
 ('5952', 4.5),
 ('5995', 3.5),
 ('6539', 3.5),
 ('6711', 4.5),
 ('7153', 4.5),
 ('7361', 5.0),
 ('8961', 4.0),
 ('33154', 4.0),
 ('33794', 4.5),
 ('44191', 4.0),
 ('44195', 4.0),
 ('48394', 4.5),
 ('50872', 4.5),
 ('54503', 4.0),
 ('55280', 4.5),
 ('55820', 4.0),
 ('58559', 5.0),
 ('59315', 5.0)

In [108]:
RecMvRDD=UserSltMvRDD.aggregateByKey((0,0),
                                      lambda a, b: (a[0]+b, a[1]+1),
                                      lambda a, b: (a[0]+b[0], a[1]+b[1])) \
                                    .map(lambda row: (row[0],row[1][0]/row[1][1]))
ResultRDD=RecMvRDD.sortBy(lambda row: row[1], ascending=False)

- connect with join

In [109]:
mvRDD=sc.textFile("movies.csv") \
    .map(lambda row: row.split(","))
mvRDD_Header=mvRDD.first()
mvRDD=mvRDD.filter(lambda x: x!=mvRDD_Header).map(lambda row: (row[0],row[1]))
# mvRDD.collect()

In [110]:
ResultRDD.join(mvRDD).map(lambda row: (row[1][1],row[1][0])).take(5)

[('Lawrence of Arabia (1962)', 5.0),
 ('"Wrestler', 5.0),
 ('Megamind (2010)', 5.0),
 ('Guardians of the Galaxy (2014)', 5.0),
 ('Fatal Instinct (1993)', 5.0)]

In [111]:
def recommend_function(givenMovieId = '31', givenUserId='1',avgRating=3.0):
    # Read in Rating records
    rtRDD=sc.textFile("ratings.csv") \
     .map(lambda row: row.split(","))
    
    # Take out header and filter first 3 columns for use
    rtRDD_Header=rtRDD.first()
    rtRDD=rtRDD.filter(lambda x: x!=rtRDD_Header).map(lambda row: (row[0],row[1],float(row[2])))
    
    # Filter distinct users with good rating on the given Movie
    GoodRtUsRDD=rtRDD.filter(lambda row: row[2]>avgRating) \
     .filter(lambda row: row[1]==givenMovieId)\
     .map(lambda row: row[0]) \
     .distinct()
    GoodRtUsLs=GoodRtUsRDD.collect()
    # Prepare User list to be Python list
    UserSltMvRDD=rtRDD.filter(lambda row: row[0] in GoodRtUsLs).map(lambda row: (row[1],row[2]))
    
    # AggregateByKey to compute average rate for each movie
    RecMvRDD=UserSltMvRDD.aggregateByKey((0,0),
     lambda a, b: (a[0]+b, a[1]+1),
     lambda a, b: (a[0]+b[0], a[1]+b[1])) \
     .map(lambda row: (row[0],row[1][0]/row[1][1]))
    ResultRDD=RecMvRDD.sortBy(lambda row: row[1], ascending=False)
    
    # Prepare Movie.csv for join
    mvRDD=sc.textFile("movies.csv") \
     .map(lambda row: row.split(","))
    mvRDD_Header=mvRDD.first()
    mvRDD=mvRDD.filter(lambda x: x!=mvRDD_Header).map(lambda row: (row[0],row[1]))
    
    # Join results with movie names
    return ResultRDD.join(mvRDD).map(lambda row: (row[1][1],row[1][0])).take(5)

print(recommend_function('7','1',2.5))

[('Crumb (1994)', 5.0), ('Three Colors: Blue (Trois couleurs: Bleu) (1993)', 5.0), ('Boys and Girls (2000)', 5.0), ('"Amazing Panda Adventure', 5.0), ('Diabolique (Les diaboliques) (1955)', 5.0)]
